In [1]:
!pip install "sagemaker>=2.48.0" "datasets==1.11" --upgrade

     |████████████████████████████████| 452 kB 43.7 MB/s eta 0:00:01
     |████████████████████████████████| 264 kB 106.1 MB/s eta 0:00:01
  Using cached xxhash-2.0.2-cp36-cp36m-manylinux2010_x86_64.whl (243 kB)
  Using cached fsspec-2021.11.0-py3-none-any.whl (132 kB)
     |████████████████████████████████| 56 kB 9.8 MB/s  eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.69.0-py2.py3-none-any.whl size=625707 sha256=a60fb1e37183482c6f8c985fbe6030539f1ff212feda0cd1b543aa3f9bdeea02
  Stored in directory: /home/ec2-user/.cache/pip/wheels/f4/2c/36/3ab4164cdc3413eb82504df7915934940baa5aa7fac923dc0f
Successfully built sagemaker
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.1.2
    Uninstalling huggingface-hub-0.1.2:
      Successfully uninstalled huggingface-hub-0.1.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2021.4.0
    Uninstalling fsspec-2021.4.0:
      Successfully uninstalled fsspec-2021.4.0
  Atte

In [3]:
!wget https://raw.githubusercontent.com/huggingface/notebooks/master/sagemaker/12_batch_transform_inference/tweet_data.csv

--2021-11-19 10:00:30--  https://raw.githubusercontent.com/huggingface/notebooks/master/sagemaker/12_batch_transform_inference/tweet_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191917 (187K) [text/plain]
Saving to: ‘tweet_data.csv’

tweet_data.csv      100%[===================>] 187.42K  --.-KB/s    in 0.003s  

2021-11-19 10:00:31 (70.8 MB/s) - ‘tweet_data.csv’ saved [191917/191917]



In [4]:
import csv
import json
import sagemaker
from sagemaker.s3 import S3Uploader,s3_path_join

# get the s3 bucket
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
sagemaker_session_bucket = sess.default_bucket()

# datset files
dataset_csv_file="tweet_data.csv"
dataset_jsonl_file="tweet_data.jsonl"

with open(dataset_csv_file, "r+") as infile, open(dataset_jsonl_file, "w+") as outfile:
    reader = csv.DictReader(infile)
    for row in reader:
        # remove @
        row["inputs"] = row["inputs"].replace("@","")
        json.dump(row, outfile)
        outfile.write('\n')

                
# uploads a given file to S3.
input_s3_path = s3_path_join("s3://",sagemaker_session_bucket,"batch_transform/input")
output_s3_path = s3_path_join("s3://",sagemaker_session_bucket,"batch_transform/output")
s3_file_uri = S3Uploader.upload(dataset_jsonl_file,input_s3_path)

print(f"{dataset_jsonl_file} uploaded to {s3_file_uri}")

tweet_data.jsonl uploaded to s3://sagemaker-us-east-1-579019700964/batch_transform/input/tweet_data.jsonl


In [5]:
from sagemaker.huggingface.model import HuggingFaceModel

# Hub Model configuration. <https://huggingface.co/models>
hub = {
    'HF_MODEL_ID':'cardiffnlp/twitter-roberta-base-sentiment',
    'HF_TASK':'text-classification'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub, # configuration for loading model from Hub
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.6", # transformers version used
   pytorch_version="1.7", # pytorch version used
   py_version='py36', # python version used
)

# create Transformer to run our batch job
batch_job = huggingface_model.transformer(
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    output_path=output_s3_path, # we are using the same s3 path to save the output with the input
    strategy='SingleRecord')

# starts batch transform job and uses s3 data as input
batch_job.transform(
    data=s3_file_uri,
    content_type='application/json',    
    split_type='Line')

......................................This is an experimental beta features, which allows downloading model from the Hugging Face Hub on start up. It loads the model defined in the env var `HF_MODEL_ID`
#015Downloading:   0%|          | 0.00/2.39k [00:00<?, ?B/s]#015Downloading: 100%|██████████| 2.39k/2.39k [00:00<00:00, 3.43MB/s]
#015Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]#015Downloading: 100%|██████████| 747/747 [00:00<00:00, 1.10MB/s]
#015Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]#015Downloading: 100%|██████████| 456k/456k [00:00<00:00, 31.1MB/s]
#015Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]#015Downloading:   1%|          | 4.39M/499M [00:00<00:11, 43.9MB/s]#015Downloading:   2%|▏         | 9.15M/499M [00:00<00:10, 46.1MB/s]#015Downloading:   3%|▎         | 14.3M/499M [00:00<00:09, 48.6MB/s]#015Downloading:   4%|▍         | 19.5M/499M [00:00<00:09, 49.8MB/s]#015Downloading:   5%|▍         | 24.6M/499M [00:00<00:09, 50.4MB/s]#015Downloading: 

In [6]:
import json
from sagemaker.s3 import S3Downloader
from ast import literal_eval
# creating s3 uri for result file -> input file + .out
output_file = f"{dataset_jsonl_file}.out"
output_path = s3_path_join(output_s3_path,output_file)

# download file
S3Downloader.download(output_path,'.')

batch_transform_result = []
with open(output_file) as f:
    for line in f:
        # converts jsonline array to normal array
        line = "[" + line.replace("[","").replace("]",",") + "]"
        batch_transform_result = literal_eval(line) 
        
# print results 
print(batch_transform_result[:3])

[{'label': 'LABEL_1', 'score': 0.766870379447937}, {'label': 'LABEL_0', 'score': 0.8912612199783325}, {'label': 'LABEL_1', 'score': 0.5760677456855774}]
